In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import io
import nest_asyncio
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai import Agent

# Fix event loop issue in Jupyter
nest_asyncio.apply()

# Ollama Model Configuration
MODEL_ID = "llama3"  # Ensure you have pulled the correct model
OLLAMA_SERVER_ENDPOINT = "http://localhost:11434/v1"

# Initialize the OpenAI-compatible model with Ollama
model_ollama = OpenAIModel(
    model_name=MODEL_ID,
    base_url=OLLAMA_SERVER_ENDPOINT,
)

# Initialize Pydantic AI Agent
agent = Agent(
    model=model_ollama,
    system_prompt="You are an AI assistant that analyzes CSV data and provides concise answers."
)

# Global variable to store uploaded CSV data
data = None

# ============================
# File Handling
# ============================
def upload_file(file):
    """Handles CSV file upload and parsing."""
    global data
    try:
        data = pd.read_csv(file.name)
        return f"✅ File uploaded successfully! Rows: {data.shape[0]}, Columns: {data.shape[1]}"
    except Exception as e:
        return f"❌ Error parsing CSV: {str(e)}"

# ============================
# Query Processing (Pydantic AI + Ollama)
# ============================
def process_query(user_query):
    """Processes user query using Pydantic AI & Ollama."""
    global data
    if data is None:
        return "⚠ Please upload a CSV file first."
    try:
        csv_string = data.to_csv(index=False)
        SAFE_LIMIT = 15000
        if len(csv_string) <= SAFE_LIMIT:
            data_for_ai = csv_string
        else:
            sample_rows = data.head(15).to_csv(index=False)
            column_info = data.describe(include="all").to_csv(index=False)
            data_for_ai = f"Sample Data:\n{sample_rows}\n\nColumn Summary:\n{column_info}"
        prompt = (
            "You are a data analyst AI. Use the given CSV data to answer the query accurately.\n"
            "If the query requires numerical computation, analyze the values correctly.\n"
            "If the query asks about patterns, trends, or correlations, provide insights based on the data.\n"
            "Do NOT just return column names; analyze the content.\n\n"
            f"Data:\n{data_for_ai}\n\nQuery: {user_query}"
        )
        ai_response = agent.run_sync(prompt)
        return f"🤖 **AI Response:** {ai_response.data}"
    except Exception as e:
        return f"❌ Error processing query: {str(e)}"

# ============================
# Gradio Interface
# ============================
with gr.Blocks() as app:
    gr.Markdown("# 📊 CSV Chatbot with Pydantic AI, LLM, and Graph Plotting")
    file_upload = gr.File(label="📂 Upload CSV", file_types=[".csv"])
    upload_btn = gr.Button("Upload")
    upload_output = gr.Textbox(label="Upload Status")
    query_input = gr.Textbox(label="💬 Ask a question about the CSV")
    query_btn = gr.Button("Submit")
    query_output = gr.Textbox(label="🤖 AI Answer")
    upload_btn.click(upload_file, inputs=[file_upload], outputs=[upload_output])
    query_btn.click(process_query, inputs=[query_input], outputs=[query_output])
app.launch()
